In [10]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:15 http://ppa.launchpa

In [11]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-24 01:31:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2022-04-24 01:31:35 (1.23 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [28]:
jdbc_url = "jdbc:postgresql://datachallenge.chmak7da6mdg.us-east-2.rds.amazonaws.com/Challenge"
vine_df = spark.read.format("jdbc").option("url",jdbc_url).option("dbtable", "vine_table").option("user", "").option("password", "").load()

In [29]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [53]:
display(vine_df)

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]

In [32]:
filter_votes_df = vine_df.filter("total_votes>20")

In [51]:
filter_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|     

In [34]:
new_filter_votes_df = filter_votes_df.filter(filter_votes_df["helpful_votes"] / filter_votes_df["total_votes"] >= 0.5)

In [35]:
new_filter_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2N92CUM7WBA1A|          5|           27|         28|   N|     

In [41]:
y_vinep_df = new_filter_votes_df.filter(new_filter_votes_df["verified_purchase"] == "Y")

In [61]:
y_vinep_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
|R2N92CUM7WBA1A|          5|           27|         28|   N|                Y|
|R2TAFZNG4KHFQB|          4|           24|         24|   N|                Y|
| RNRFACS48ZE4A|          5|           21|         21|   N|     

In [43]:
n_vinep_df = new_filter_votes_df.filter(new_filter_votes_df["verified_purchase"] == "N")

In [44]:
n_vinep_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2PO5QTLXJ9LHG|          5|           33|         46|   N|                N|
|R33BKAVFS3HJ97|          5|           22|         23|   N|                N|
|R2QTJNPQ8TS8NB|          5|           22|         22|   N|                N|
|R2DOHUQGQG3VNN|          1|           93|        102|   N|                N|
|R2ZK070EW5D3B8|          5|           46|         48|   N|                N|
|R2S8TIFNDTKLTT|          4|           13|         21|   N|                N|
| RBKGYFNAEOD7F|          1|           16|         29|   N|                N|
|R1G9FY5L84IAOG|          1|           32|         34|   N|     

In [71]:
total_star_y = y_vinep_df.count()
star5_paid = y_vinep_df.filter(y_vinep_df["star_rating"] == 5).count()
n_star5_paid = y_vinep_df.filter(y_vinep_df["star_rating"] != 5).count()
star5_per = (star5_paid / total_star_y) * 100
print('The total paid reviews are:' + str(total_star_y),
      'the total 5 star paid reviews are:' + str(star5_paid),
      'the total no 5 star paid reviews are:' + str(n_star5_paid), 
      'and the percentage of 5 star paid reviews of the total reviews are:' + str(round(star5_per, 2)))

The total paid reviews are:4814 the total 5 star paid reviews are:2502 the total no 5 star paid reviews are:2312 and the percentage of 5 star paid reviews of the total reviews are:51.97


In [73]:
total_star_n = n_vinep_df.count()
star5_unpaid = n_vinep_df.filter(n_vinep_df["star_rating"] == 5).count()
n_star5_unpaid = n_vinep_df.filter(n_vinep_df["star_rating"] != 5).count()
un_star5_per = (star5_unpaid / total_star_n) * 100
print('The total unpaid reviews are:' + str(total_star_n),
      'the total 5 star unpaid reviews are:' + str(star5_unpaid),
      'the total no 5 star unpaid reviews are:' + str(n_star5_unpaid), 
      'and the percentage of 5 star unpaid reviews of the total reviews are:' + str(round(un_star5_per, 2)))

The total unpaid reviews are:2979 the total 5 star unpaid reviews are:1539 the total no 5 star unpaid reviews are:1440 and the percentage of 5 star unpaid reviews of the total reviews are:51.66
